In [ ]:
!pip install transformers datasets rouge sentencepiece

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install transformers[torch]

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re
import torch

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
from torch.utils.data import Dataset
from transformers import AutoTokenizer, TrainingArguments, Trainer

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
DF_PATH = '/kaggle/input/tweetsumfixed/tweets_df_new.csv'
df = pd.read_csv(DF_PATH)

In [7]:
df

,text
0,@115712 I understand. I would like to assist y...
1,@sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...
3,@115712 Please send us a Private Message so th...
4,@sprintcare I did.
...,...
2800590,"@823869 Hey, we'd be happy to look into this f..."
2800591,@115714 wtf!? I’ve been having really shitty s...
2800592,@143549 @sprintcare You have to go to https://...
2800593,"@823870 Sounds delicious, Sarah! 😋 https://t.c..."


### Предобработка твитов

In [8]:
def preprocess_text(text):
    text = re.sub(r"@(\w){1,20}", '', text)
    text = re.sub(r"http[\w\:\/\\\.]+" ,'', text)
    text = re.sub(r"\<[\w]+\>" ,'', text)
    return " ".join(text.split())

In [9]:
df['text'] = df['text'].apply(preprocess_text)
df = df.drop_duplicates().dropna()

In [10]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

### Модель

In [19]:
tokenizer_name = "bert-base-uncased"
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

In [20]:
def preprocess_function(inputs):

    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    return model_inputs

In [21]:
preprocess_function(['hello'])

{'input_ids': [[101, 7592, 102]], 'token_type_ids': [[0, 0, 0]], 'attention_mask': [[1, 1, 1]]}

In [22]:
class TorchDataset(Dataset):

    def __init__(self, df):
        self.df = df

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        return preprocess_function(self.df.loc[idx, 'text'])

In [23]:
train_dataset = TorchDataset(train_df)
test_dataset = TorchDataset(test_df)

In [25]:
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained(model_name)
model.to(device)

config.json:   0%|          | 0.00/664 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [26]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = '[PAD]'
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [27]:
training_args = TrainingArguments(
    output_dir="tweet-bert-uncased-2",
    save_strategy="steps",
    evaluation_strategy="steps",
    eval_steps=50000,
    save_steps=50000,
    learning_rate=1e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
    load_best_model_at_end=True,
    report_to='none'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator
)

trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
50000,1.671900,1.889556
100000,1.642200,1.880300
150000,1.661900,1.866874
200000,1.699900,1.854465
250000,1.622900,1.845560
300000,1.625600,1.827603


KeyboardInterrupt: 